# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import math
from math import sin, cos, asin, pi, sqrt
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import numpy as np
import warnings
import re
warnings.filterwarnings('ignore')

In [2]:
# constants we need for processing data:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
UBER_CSV = "uber_rides_sample.csv"




NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_Using the latituede/longitude given in the Yellow Taxi Dataframe to calculate the route distance._

In [3]:
def calculate_distance(from_coord, to_coord):
    """
    Calculate the distance between two coordinates
    
    The function takes in 2 argumants: "from_coord" and "to_coord"
    "from_coord": columns 'pickup_latitude' and 'pickup_longitude' from datarame
    "to_coord":  columns 'dropoff_latitude' and 'dropoff_longitude' from datarame
    
    RETURN: an array of calculated distances (in km)
    """
    # first change the latitude and longitude into radians
    lat1 = from_coord['pickup_latitude'] *pi/180
    lon1 = from_coord['pickup_longitude'] *pi/180
    lat2 = to_coord['dropoff_latitude']*pi/180
    lon2 = to_coord['dropoff_longitude']*pi/180
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    
    # using Haversine formula to calculate distance
    a = ((dlat/2).map(sin))**2 + (lat1.map(cos)) * (lat2.map(cos)) * ((dlon/2).map(sin))**2
    
    # suppose the radius of earth is 6371 km
    distance = 6371*2 * (a.map(sqrt)).map(asin)
    
    # round the distance with 2 decimals
    return round(distance, 2)

In [4]:
def add_distance_column(dataframe):
    """
    Add one column 'distance' which calculates taxi route distance into 'dataframe'
    
    The function takes one argument: 'dataframe'
    Use columns "pickup_latitude" and "pickup_longitude" to create argument "from_coord"
    Use columns "dropoff_latitude" and "dropoff_longitude" to create argument to_coord"
    
    RETURN: dataframe with new column 'distance' 
    """
    # generate 'from_coord' & 'to_coord'
    from_coord = dataframe[['pickup_latitude', 'pickup_longitude']]
    to_coord = dataframe[['dropoff_latitude', 'dropoff_longitude']]
    
    # add 'distance' into dataframe
    dataframe['distance'] = calculate_distance(from_coord, to_coord)
    
    return dataframe

### Processing Taxi Data

_Obtain Yellow Taxi Data from 2009-01 to 2015-06, then clean and combine all dataframes._

In [5]:
# Load shp file of Taxi Zones
import geopandas as gpd
g = gpd.read_file(filename='taxi_zones.zip', engine='fiona')
g = g.to_crs(4326)

# use the center of the zones as a point to look up the coordinates.
g['longitude'] = g.centroid.x
g['latitude'] = g.centroid.y

# remove unnecessary columns 
# create mapping methods for latitude & longitude
# aim to create lat & longitude which do not exist in some taxi data
g = g[['LocationID', 'longitude', 'latitude', 'zone', 'borough']]
lat_map = dict(zip(g['LocationID'], g['latitude']))
lon_map = dict(zip(g['LocationID'], g['longitude']))

In [6]:
def find_taxi_csv_urls():
    """
    Get all the URLs of Taxi Data
    RETURN: a lits of links that download parquet files of Taxi Data
    """
    y = []
    links = []
    # get response from HTML
    response = requests.get(TAXI_URL)
    html = response.content
    
    # use bs4 to parse html and find Yellow Taxi urls
    soup = bs4.BeautifulSoup(html, "html.parser")
    yellow = soup.find_all("a", attrs = {"title":"Yellow Taxi Trip Records"})
    y.append([a["href"] for a in yellow])
    
    # use regex pattern to find desired URLs from 'yellow taxi trip records'
    pattern = re.compile(".*(2009|201[0-4]|2015-0[1-6]).*")
    for i in y[0]:
        match = re.match(pattern, i)
        if match != None:
            links.append(i)
    return links

In [9]:
def get_and_clean_month_taxi_data(url):
    """
    The function takes in one url from Internet
    Obtain dataframe from url of every given month and clean the df
    
    RETURN: a cleaned dataframe
    """   
    
    df = pd.read_parquet(url)
    
    # process dataframe which only has LocationIDs 
    # match the LocationID in 'df' with dataframe 'g'
    # add 4 new columns to 'df' after matching
    if 'PULocationID' in df.columns:
        df['pickup_latitude']  = df['PULocationID'].map(lat_map)
        df['pickup_longitude'] = df['PULocationID'].map(lon_map)
        df['dropoff_latitude'] = 
        df['DOLocationID'].map(lat_map)
        df['dropoff_longitude']= df['DOLocationID'].map(lon_map)

    # normalizing column names, according to UBER_DATA column names
    df = df.rename(columns={'tpep_pickup_datetime':'pickup_datetime', 
                            'Trip_Pickup_DateTime':'pickup_datetime', 
                            'Fare_Amt':'fare_amount',
                            'Start_Lon':'pickup_longitude','Start_Lat':'pickup_latitude', 
                            'End_Lon':'dropoff_longitude', 'End_Lat':'dropoff_latitude',           
                            })
    
    # remove unnecessary columns, only keep 6 columns we need
    df = df[['pickup_datetime','fare_amount',
             'pickup_latitude','pickup_longitude',
             'dropoff_latitude','dropoff_longitude',
            ]]
    
    # using appropriate column type
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df.iloc[:,1:6] = df.iloc[:,1:6].astype('float32')
        
    # remove invalid ponits
    df = df[df['fare_amount']>0]
        
    # removing trips that start and/or end outside of the BOX
    df = df[df['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
        
    df = df[df['dropoff_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['dropoff_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['dropoff_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['dropoff_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
    
    # sampling from data to ensure the amount of taxi_data matches that of uber_data 
    df = df.sample(n=round(195454/78))
     
    # print 'done' after cleaning & return cleaned monthly data
    file = url.split('/')[-1]
    print(f'done with {file}')
    
    return df

In [11]:
def get_and_clean_taxi_data():
    """
    The function generate a list of Yellow Taxi urls
    For each url, obtain the dataframe from url
    Then clean dataframe and add 'distance' column
    In the end we join each month's dataframe to a giant one
    
    RETURN: a huge dataframe contains cleaned data from every month
    
    """
    all_taxi_dataframes = []
    
    # acquire list of Yellow Taxi urls
    all_parquet = find_taxi_csv_urls()
    
    # clean dataframe and add columns
    for i in all_parquet:
        
        # read local files by extracting file name from url
        file = i.split('/')[-1]
        dataframe = get_and_clean_month_taxi_data(file)
        add_distance_column(dataframe)  
        all_taxi_dataframes.append(dataframe)
    
    # combine to a giant dataframe
    taxi_data = pd.concat(all_taxi_dataframes)
    
    return taxi_data

In [13]:
%%time
get_and_clean_taxi_data()

done with yellow_tripdata_2015-01.parquet
done with yellow_tripdata_2015-02.parquet
done with yellow_tripdata_2015-03.parquet
done with yellow_tripdata_2015-04.parquet
done with yellow_tripdata_2015-05.parquet
done with yellow_tripdata_2015-06.parquet
done with yellow_tripdata_2014-01.parquet
done with yellow_tripdata_2014-02.parquet
done with yellow_tripdata_2014-03.parquet
done with yellow_tripdata_2014-04.parquet
done with yellow_tripdata_2014-05.parquet
done with yellow_tripdata_2014-06.parquet
done with yellow_tripdata_2014-07.parquet
done with yellow_tripdata_2014-08.parquet
done with yellow_tripdata_2014-09.parquet
done with yellow_tripdata_2014-10.parquet
done with yellow_tripdata_2014-11.parquet
done with yellow_tripdata_2014-12.parquet
done with yellow_tripdata_2013-01.parquet
done with yellow_tripdata_2013-02.parquet
done with yellow_tripdata_2013-03.parquet
done with yellow_tripdata_2013-04.parquet
done with yellow_tripdata_2013-05.parquet
done with yellow_tripdata_2013-06.

,pickup_datetime,fare_amount,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,distance
12444548,2015-01-31 13:10:47,8.500000,40.759819,-73.984200,40.756729,-73.965149,1.64
3446142,2015-01-09 20:29:32,5.000000,40.758030,-73.977699,40.768616,-73.965637,1.56
8989985,2015-01-22 15:44:58,16.500000,40.747746,-73.978493,40.775932,-73.946510,4.13
10025775,2015-01-24 19:38:05,3.500000,40.747746,-73.978493,40.756729,-73.965149,1.50
8195640,2015-01-20 18:10:23,10.000000,40.768616,-73.965637,40.768616,-73.965637,0.00
...,...,...,...,...,...,...,...
11780603,2009-12-27 23:05:45,21.700001,40.774155,-73.872902,40.744255,-73.991905,10.56
7351525,2009-12-12 22:18:53,7.300000,40.737209,-73.989014,40.721149,-73.993698,1.83
5126054,2009-12-11 16:31:57,5.300000,40.740437,-73.995132,40.751232,-73.984299,1.51
12806072,2009-12-19 10:23:09,11.700000,40.700745,-73.987869,40.748356,-73.996391,5.34


### Processing Uber Data

_Processing the Uber Data, make the column type and column names the same pattern as Taxi Data._

In [34]:
def load_and_clean_uber_data(csv_file):
    """
    The funcion takes one argument, the csv file
    We do the same procedure as cleaning Taxi Data
    We aim to obtain dataframe in the same colmn type as Taxi Data
    
    RETURN: a cleaned dataframe
    """
    df = pd.read_csv(csv_file)
    
    # remove unnecessary columns
    df = df[['pickup_datetime','fare_amount',
             'pickup_latitude','pickup_longitude',
             'dropoff_latitude','dropoff_longitude',
            ]]
    
    # removing trips that start and/or end outside of the BOX
    df = df[df['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
        
    df = df[df['dropoff_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['dropoff_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['dropoff_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['dropoff_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
    
    # using appropriate column type
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df.iloc[:,1:6] = df.iloc[:,1:6].astype('float32')
        
    # remove invalid ponits
    df = df[df['fare_amount']>0]
    
    return df

In [35]:
def get_uber_data():
    """
    The function loads and cleans Uber_data
    Calculate and Add 'distance' column to uber dataframe
    
    RETURN: cleaned dataframe with new column 'distance'
    """
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [29]:
def clean_month_weather_data_hourly(csv_file):
    
    #read the csv file 
    df=pd.read_csv(csv_file)
    
    #clean the data
    
    #remove all columns that contain the world 'Daily'
    df=df.drop(df.filter(regex='Daily').columns, axis=1)
    
    #remove all columns that have all NaN values
    df=df.dropna(axis=1, how="all")
    
    #remove all columns with only zero values
    df.loc[:, (df != 0).any(axis=0)]
    
    
    return df
    

In [31]:
def clean_month_weather_data_daily(csv_file):
     
    #read the csv file 
    df=pd.read_csv(csv_file)
    
    #clean the data
    
    #remove all columns that contain the world 'Hourly'
    df=df.drop(df.filter(regex='Hourly').columns, axis=1)
    
    #remove all columns that have all NaN values
    df=df.dropna(axis=1, how="all")
    
    #remove all columns with only zero values
    df.loc[:, (df != 0).any(axis=0)]
    
    
    return df

In [33]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    
    #comprehension list to generate all the csv files name needed
    weather_csv_files = [f"{year}_weather.csv" for year in range(2009,2016)]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [40]:
#TEST CELL

hourly_data, daily_data=load_and_clean_weather_data()

#show all columns
pd.set_option('display.max_columns', None)

hourly_data

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,SOURCE,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,Sunrise,Sunset,REM,BackupDirection,BackupDistance,BackupDistanceUnit,BackupElements,BackupEquipment,BackupName,WindEquipmentChangeDate,HourlySkyConditions,HourlyStationPressure,HourlyWetBulbTemperature,MonthlyDaysWithGT001Precip,MonthlyDaysWithGT010Precip,MonthlyDaysWithGT32Temp,MonthlyDaysWithGT90Temp,MonthlyDaysWithLT0Temp,MonthlyDaysWithLT32Temp,MonthlyDepartureFromNormalAverageTemperature,MonthlyDepartureFromNormalCoolingDegreeDays,MonthlyDepartureFromNormalHeatingDegreeDays,MonthlyDepartureFromNormalMaximumTemperature,MonthlyDepartureFromNormalMinimumTemperature,MonthlyDepartureFromNormalPrecipitation,MonthlyGreatestPrecip,MonthlyGreatestPrecipDate,MonthlyGreatestSnowDepth,MonthlyGreatestSnowDepthDate,MonthlyGreatestSnowfall,MonthlyGreatestSnowfallDate,MonthlyMaxSeaLevelPressureValue,MonthlyMaxSeaLevelPressureValueDate,MonthlyMaxSeaLevelPressureValueTime,MonthlyMaximumTemperature,MonthlyMeanTemperature,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureValueDate,MonthlyMinSeaLevelPressureValueTime,MonthlyMinimumTemperature,MonthlySeaLevelPressure,MonthlyStationPressure,MonthlyTotalLiquidPrecipitation,MonthlyTotalSnowfall,AWND,CDSD,CLDD,DSNW,HDSD,HTDD,NormalsCoolingDegreeDay,NormalsHeatingDegreeDay,ShortDurationEndDate005,ShortDurationEndDate010,ShortDurationEndDate015,ShortDurationEndDate020,ShortDurationEndDate030,ShortDurationEndDate045,ShortDurationEndDate060,ShortDurationEndDate080,ShortDurationEndDate100,ShortDurationEndDate120,ShortDurationEndDate150,ShortDurationEndDate180,ShortDurationPrecipitationValue005,ShortDurationPrecipitationValue010,ShortDurationPrecipitationValue015,ShortDurationPrecipitationValue020,ShortDurationPrecipitationValue030,ShortDurationPrecipitationValue045,ShortDurationPrecipitationValue060,ShortDurationPrecipitationValue080,ShortDurationPrecipitationValue100,ShortDurationPrecipitationValue120,ShortDurationPrecipitationValue150,ShortDurationPrecipitationValue180
0,72505394728,2009-01-01T00:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.01,3.0,18.0,NaN,NaN,-0.04,3.0,53.0,29.99,10.0,290,37.0,18.0,NaN,NaN,MET137METAR KNYC 010551Z AUTO 29016G32KT 260V3...,ESE,0.5,mi,SNOW,SNOWBOARD,CENTRAL PARK ZOO,2006-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72505394728,2009-01-01T01:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.03,3.0,18.0,NaN,NaN,NaN,NaN,53.0,30.01,10.0,290,28.0,18.0,NaN,NaN,MET108METAR KNYC 010651Z AUTO 29016G24KT 10SM ...,ESE,0.5,mi,SNOW,SNOWBOARD,CENTRAL PARK ZOO,2006-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72505394728,2009-01-01T02:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.07,3.0,18.0,NaN,NaN,NaN,NaN,53.0,30.05,10.0,290,29.0,18.0,NaN,NaN,MET108METAR KNYC 010751Z AUTO 29016G25KT 10SM ...,ESE,0.5,mi,SNOW,SNOWBOARD,CENTRAL PARK ZOO,2006-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72505394728,2009-01-01T03:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.09,3.0,18.0,NaN,Na

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,

);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATE,
    pickup_longitude FLOAT32,
    pickup_latitude FLOAT32,
    dropoff_longitude FLOAT32,
    dropoff_latitude FLOAT32,
    fare_amount FLOAT32,
    distance FLOAT32
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATE,
    pickup_longitude FLOAT32,
    pickup_latitude FLOAT32,
    dropoff_longitude FLOAT32,
    dropoff_latitude FLOAT32,
    fare_amount FLOAT32,
    distance FLOAT32
);

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)